In [1]:
import csv, re
f = open('ExpertInterview.csv')
reader = csv.reader(f)
codes = list(reader)

def addToCodeBook(codes):
    all_codes.update(codes)

def cleanCodes(rawCodes):
    rawCodes = re.split(' |,|\n', rawCodes)
    codes = [code.lower().strip() for code in rawCodes if code]
    addToCodeBook(codes)
    return codes

In [2]:
quoteDict = {}
all_codes = set({})
for index, code in enumerate(codes[1:]): 
    if not (code[1] == '' and code[2] == ''):
        quote = {}
        quote['quote'] = code[0]
        quote['chris'] = cleanCodes(code[1])
        quote['alina'] = cleanCodes(code[2])
        quoteDict[index] = quote

In [ ]:
all_codes

In [ ]:
# quoteDict

In [3]:
# Preparing codes for NLTK
from collections import defaultdict

codeDict = defaultdict(dict)
for code in all_codes: 
    for ix, quote in quoteDict.items(): 
        quotes = {}
        quotes['alina'] = 1 if code in quote['alina'] else 0
        quotes['chris'] = 1 if code in quote['chris'] else 0 
        quotes['quote'] = quote['quote']
        codeDict[code][ix] = quotes

In [ ]:
# codeDict["#social"]

In [4]:
# Provide a single tag, or multiple tags. 
raters = ['alina', 'chris']
raterFlag = [False] * len(raters)

def getAgreementCodes(tags):
    data = []
    for rater in raters: 
        for itemlabel, quote in quoteDict.items():
            if len(tags) > 1 and all([tag in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#Both"])
            if all([tag not in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#None"])
            else: 
                for tag in tags:
                    if tag in quote[rater]:
                        data.append([rater, itemlabel, tag])
    return data

In [5]:
# Sets of labels not needed for this dataset. 
# labels = [
#     ("#LackingSupport", "#ReceivingSupport"),
#     ("#Believable", "#NotBelievable"),
#     ("#MiddleGround", "#Disagreement"),
#     ("#StandingGround", "#ChangedOpinion"),
#     ("#Expected", "#Unexpected"),
#     ("#DecreasingCertainty", "#IncreasingCertainty"),
#     ("#UncertainMindsChange", "#ChangedDespiteCertainty"),
#     ("#InfluenceGroup",),
#     ("#InfluenceIndividual",),
#     ("#InfluenceArticle",),
#     ("#ReinforcedViews",),
#     ("#NPCMentionedUnprompted",),
#     ("#UsedPoliticalAffiliationSteriotype",),
#     ("#IdentifyingSimilarGroups",),
#     ("#SimilarViewsConverge",),
#     ("#OpinionAttitudeDifference",),
#     ("#CertaintyConvinces",),
#     ("#PoliticalIdentity",),
#     ("#NotEnoughInfo",),
#     ("#EmotionalResponse",),
#     ("#InferFactsFrom",),
#     ("#CausalInference",),
#     ("#Meta",),
#     ("#GroupsStandGround",),
#     ("#NoAnswer",),
#     ("#Polarization",),
#     ("#Infer",),
#     ("#AgreeWithMajority",),
#     ("#ExtremeChangesInOpinion",),
#     ("#OpinionAttitudeAlignment",),
#     ("#LimitedKnowledge",),
#     ("#OvertonWindow",)
# ]
all_codes = [(label,) for label in all_codes]
all_codes

[('#active',),
 ('#contractofcare',),
 ('#multisensory',),
 ('#marketing',),
 ('#theme',),
 ('#digital',),
 ('#interactionengineering',),
 ('#audienceperforming',),
 ('#presence',),
 ('#narrativecomplexity',),
 ('#space',),
 ('#attention',),
 ('#moodengineering',),
 ('#publicprivate',),
 ('#narrative',),
 ('#experiencespace',),
 ('#companygoals',),
 ('#immersion',),
 ('#tech',),
 ('#advantage',),
 ('#time',),
 ('#inspiration',),
 ('#interaction',),
 ('#spectrum',),
 ('#authorcreatorgoaltension',),
 ('#discoverymoment',),
 ('#goals',),
 ('#physicallimitation',),
 ('#personalization',),
 ('#audienceexpectation',),
 ('#connection',),
 ('#novelty',),
 ('#culture',),
 ('#disadvantage',),
 ('#process',),
 ('#creatingmagic',),
 ('#mr',),
 ('#engagement',),
 ('#interactiondisadvantages',),
 ('#sensory',),
 ('#storytelling',),
 ('#cues',),
 ('#morethanspectrum',),
 ('#socialengineering',),
 ('#transmedia',),
 ('#outcomes',),
 ('#visualdesign',),
 ('#narrativegoal',),
 ('#authoringgoal',),
 ('#t

In [9]:
from nltk.metrics.agreement import AnnotationTask
from nltk.probability import FreqDist

with open('irr_calculations.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["Tag", "Kappa", "Fleiss", "Alpha", "Scotts"])

    total_fleiss = []
    total_kappa = []
    total_alpha = []
    total_scotts = []

    for code in all_codes: 
        nltk_code_format = getAgreementCodes(code)
        task = AnnotationTask(data=nltk_code_format)
        label_freqs = FreqDist(x['labels'] for x in task.data)
        print("Codes: ", label_freqs.keys())
        try: 
            kappa = str(task.kappa())
            total_kappa.append(task.kappa())
            fleiss = task.multi_kappa()
            total_fleiss.append(task.multi_kappa())
            alpha = str(task.alpha())
            total_alpha.append(task.alpha())
            scotts = str(task.pi())
            total_scotts.append(task.pi())
            csvwriter.writerow([label_freqs.keys(), kappa, fleiss, alpha, scotts])
            print([label_freqs.keys(), kappa, fleiss, alpha, scotts])

        except ZeroDivisionError as e: 
            print("Expected Agreement: 1.0 - ZeroDivisionError")

    tot_fleiss = sum(total_fleiss)/len(total_fleiss)
    tot_kappa = sum(total_kappa)/len(total_kappa)
    tot_alpha = sum(total_alpha)/len(total_alpha)
    tot_scotts = sum(total_scotts)/len(total_scotts)
    csvwriter.writerow(["Total", tot_kappa, tot_fleiss, tot_alpha, tot_scotts])
    print(["Total", tot_kappa, tot_fleiss, tot_alpha, tot_scotts])

Codes:  dict_keys(['#None', '#active'])
[dict_keys(['#None', '#active']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#contractofcare'])
[dict_keys(['#None', '#contractofcare']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#multisensory'])
[dict_keys(['#None', '#multisensory']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#marketing'])
[dict_keys(['#None', '#marketing']), '0.7897959183673467', 0.7897959183673467, '0.7908163265306123', '0.789795918367347']
Codes:  dict_keys(['#None', '#theme'])
[dict_keys(['#None', '#theme']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#digital'])
[dict_keys(['#None', '#digital']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#interactionengineering'])
[dict_keys(['#None', '#interactionengineering']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#audienceperforming'])
[dict_keys(['#None', '#audienceperforming']), '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#presence'])
[dict_key